In [1]:
import numpy as np
import pandas as pd

In [2]:
coliDf = pd.read_csv("../data/collisions_sd.csv")
coliDf

,report_id,date_time,police_beat,address_no_primary,address_pd_primary,address_road_primary,address_sfx_primary,address_pd_intersecting,address_name_intersecting,address_sfx_intersecting,violation_section,violation_type,charge_desc,injured,killed,hit_run_lvl
0,171111,2015-01-14 20:00:00,835,4200,,JUNIPER,STREET,,,,MISC-HAZ,VC,MISCELLANEOUS HAZARDOUS VIOLATIONS OF THE VEHI...,0,0,MISDEMEANOR
1,192016,2015-03-19 12:00:00,622,5200,,LINDA VISTA,ROAD,,,,MISC-HAZ,VC,MISCELLANEOUS HAZARDOUS VIOLATIONS OF THE VEHI...,0,0,MISDEMEANOR
2,190012,2015-03-24 03:05:00,626,1000,W,WASHINGTON,STREET,,,,22107,VC,TURNING MOVEMENTS AND REQUIRED SIGNALS,2,0,NaN
3,191866,2015-03-27 23:56:00,613,2800,,WORDEN,STREET,,,,22107,VC,TURNING MOVEMENTS AND REQUIRED SIGNALS,1,0,NaN
4,185207,2015-07-06 11:45:00,813,2800,,EL CAJON,BOULEVARD,,,,20002(A),VC,HIT AND RUN,0,0,MISDEMEANOR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64554,23022608,2023-05-27 05:10:00,311,3200,,WEST CANYON,AVENUE,,,,23152A,VC,DUI ALCOHOL (M),1,0,NaN
64555,23202943,2023-05-27 12:31:00,447,5000,,FEDERAL,BOULEVARD,,,,20002(A),VC,HIT AND RUN:PROP DAMAGE (M),0,0,MISDEMEANOR
64556,23202949,2023-05-27 14:15:00,524,900,,11TH,AVENUE,,,,22350,VC,UNSAFE SPEED (BASIC SPEED LAW) (I),0,0,NaN
64557,23202956,2023-05-27 21:05:06,811,4300,,CHEROKEE,AVENUE,,,,22107,VC,TURNS:UNSAFE TURN AND/OR NO TURN SIGNAL (I),0,0,MISDEMEANOR


In [4]:
import redis
import time

# Connect to the Redis container
r = redis.Redis(host='localhost', port=6379, db=0)

r.hset('fact_memory', mapping={0: 1, 1: 1, 'max': 1})


In [7]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [12]:
from redis.commands.json.path import Path
from redis.commands.search.field import TextField, NumericField, TagField
from redis.commands.search.query import NumericFilter, Query
from redis.commands.search.indexDefinition import IndexDefinition, IndexType

In [13]:
r.flushall()

schema = (
    TextField("$.state", as_name="state"),
    TextField("$.last_name", as_name="lname"),
    TextField("$.first_name", as_name="finame"), 
    TextField("$.full_name", as_name="funame"),
    TextField("$.party", as_name="party"),  
    TextField("$.twitter", as_name="twt_id"),
    TextField("$.facebook", as_name="fb_id"),
    TextField("$.youtube", as_name="yt_id"),
    TextField("$.youtube_id", as_name="ytid_id")
)

rs = r.ft("idx:persons")
rs.create_index(
    schema,
    definition=IndexDefinition(
        prefix=["person_"], index_type=IndexType.JSON
    )
)

b'OK'

In [14]:
pipe = r.pipeline()
groups = legislators_df.shape[0] // 10
schema_keys = ["last_name", "first_name", "full_name", "party", 
               "state", "twitter", "facebook", "youtube", "youtube_id"]

start_time = time.time()
for index, row in tqdm(legislators_df.iterrows(), total=legislators_df.shape[0]):
    # pipe.hset(f"person_{index}", mapping=row.to_dict())
    temp_dict = {k: v for k, v in row.to_dict().items() if k in schema_keys}
    # r.json().set(f"person_{index}", Path.root_path(), temp_dict)
    pipe.json().set(f"person_{index}", Path.root_path(), temp_dict)
    if (index+1) % groups == 0:
        pipe.execute()
        pipe = r.pipeline()

# Executing leftover pipeline commands
pipe.execute()

100%|██████████| 537/537 [00:00<00:00, 3477.73it/s]


[True, True, True, True, True, True, True]

In [15]:
# Check if redis is populated
res = rs.search(
    Query("Baldwin")#.return_field("$.party", as_field="party")
).docs
res

[Document {'id': 'person_34', 'payload': None, 'json': '{"last_name":"Baldwin","first_name":"Tammy","full_name":"Tammy Baldwin","state":"WI","party":"Democrat","twitter":"SenatorBaldwin","facebook":"senatortammybaldwin","youtube":"witammybaldwin","youtube_id":"UC_XjYCRbbI2_TDDjJidwk0Q"}'}]

In [23]:
def question_2b(state, rs, first_few=10):
    raw = rs.search(
        Query(f"@state:{state}", first_few)
    )
    res = raw.docs
    total = raw.total
    convert = [json.loads(d["json"]) for d in res]
    out = [{"name": d["first_name"] + " " + d["last_name"], 
           "party": d["party"]}
           for d in convert]
    return {"total": total, "first_few": out}

In [25]:
legislators_df[legislators_df.state=="NY"].shape[0]

29

In [26]:
question_2b("NY", rs, first_few=15)

{'total': 29,
 'first_few': [{'name': 'Chris Jacobs', 'party': 'Republican'},
  {'name': 'Kirsten Gillibrand', 'party': 'Democrat'},
  {'name': 'Yvette Clarke', 'party': 'Democrat'},
  {'name': 'Eliot Engel', 'party': 'Democrat'},
  {'name': 'Brian Higgins', 'party': 'Democrat'},
  {'name': 'Peter King', 'party': 'Republican'},
  {'name': 'Nita Lowey', 'party': 'Democrat'},
  {'name': 'Carolyn Maloney', 'party': 'Democrat'},
  {'name': 'Gregory Meeks', 'party': 'Democrat'},
  {'name': 'Jerrold Nadler', 'party': 'Democrat'},
  {'name': 'Tom Reed', 'party': 'Republican'},
  {'name': 'Charles Schumer', 'party': 'Democrat'},
  {'name': 'José Serrano', 'party': 'Democrat'},
  {'name': 'Nydia Velázquez', 'party': 'Democrat'},
  {'name': 'Grace Meng', 'party': 'Democrat'}]}